I discovered the Elf Game as a maths teacher, a fun Christmas activity with a competitive edge. Players run a business where they make money by sending elves out to cut down christmas trees and sell them. Risks are associated with different tree locations, but with a greater reward. Players also have the opportunity to hire more elves. After being soundly beaten by some students, I decided to take revenge and solve the elf game. 

In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
%matplotlib inline

/Users/Home/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
# Weather is either good or bad, with a 2:1 ratio.
def weather():
    weather = np.random.randint(1,7)
    if weather < 5:
        weather_type = 'Good'
    else:
        weather_type = 'Bad'
    return weather_type

# Woods Profit
# Trees from the woods always return a $10 price per tree
def woods_profit(elf_number, weather):
    profit = elf_number*10
    return profit

# Forest profit
# Trees from the forest return a $20 price per tree, if the weather is good. In bad weather, no trees are harvested.
def forest_profit(elf_number, weather):
    profit = 0
    if weather == 'Good':
        profit = elf_number*20
    else: 
        profit = 0
    return profit
        
# Mountains_profit
# Trees from the Mountain return a $50 price per tree, if the weather is good. 
# In bad weather, no trees are harvested, and any elves on the mountain die.
def mountain_profit(elf_number, weather):
    profit = 0
    if weather == 'Good':
        profit = elf_number*50
    else:
        profit = 0
    return profit

# Elf count changers

#Elves can be hired at $75 per elf

def mountain_loss(elf_number, weather_type):
    if weather_type == 'Bad':
        lost_elves = elf_number
    else:
        lost_elves = 0
    return lost_elves

def elf_hire(elf_number=0):
    hire_cost = 75*elf_number
    return hire_cost

def elf_count_adjuster(current_elf_total, mountain_elves, elf_hire_number, weather_type):
    new_elf_total = current_elf_total + elf_hire_number - mountain_loss(mountain_elves, weather_type)
    return new_elf_total
    
    

# Day profit
def day_profit(weather_type, wood_elves, forest_elves, mountain_elves, elf_hire_number):
    weather_today = weather_type
    profit = woods_profit(wood_elves, weather_type) + forest_profit(forest_elves, weather_type) \
             + mountain_profit(mountain_elves, weather_type) - elf_hire(elf_hire_number)
    return profit


    



In [3]:
## User input to record strategy for elf assignments to the woods, forest and mountains, 
# returning a profit history at the end of the game

# Each day, player must make a Day_plan
def day_plan(elf_total, elf_hire_max=0):
    print """You must decide how many elves should go to work in the woods, and how many in the Forest.
           Remaining elves will be sent to the mountain!"""
    print "You currently employ %s elves" %elf_total
    elf_hire_number = input("How many elves do you want to hire?")
    while elf_hire_number > elf_hire_max:
        print """You can't afford to hire that many elves! Each elf costs £75 to hire.
                 You are allowed to hire %s elves""" %elf_hire_max
        elf_hire_number = input("How many elves do you want to hire?")
    
    wood_elf_count = input("How many elves should go to the woods?")
    forest_elf_count = input("How many elves should go to the forest?")
    
    mountain_elf_count = elf_total + elf_hire_number - wood_elf_count - forest_elf_count
    
    if wood_elf_count + forest_elf_count > elf_total + elf_hire_number:
        print "You can't send more elves out to work than you employ!"
        wood_elf_count = forest_elf_count = mountain_elf_count = 0
        
    
    return wood_elf_count, forest_elf_count, mountain_elf_count, elf_hire_number

## Day_plan is used within the game to generate the next state for profit and elf count
def human_elf_game():
    day_count = 1
    total_profit = 0
    profit_tracker = []
    elf_total = 12
    while day_count < 10:
        print "Today is day %s" %day_count
        print "You have %s elves" %elf_total
        elf_hire_max = np.floor(total_profit/75)
    
        plan = (0,)*4
        while plan == (0,)*4:
            plan = day_plan(elf_total, elf_hire_max)
            if elf_total<1:
                print "You've got no elves. Make sure you hire some!"
                break
        print plan
        weather_today = weather()
        print "The weather today is %s" %weather_today
        total_profit += day_profit(weather_today, *plan)
        print "So far you have made %s" %(total_profit)
        elf_total = elf_count_adjuster(elf_total, plan[2], plan[3], weather_today)
        day_count+=1
        
        
        profit_tracker.append(total_profit)
    return profit_tracker
    


In [4]:
## My first attempt to automate the elf game was to write a function which generated a random plan for each day

def random_plan_generator(elf_count, day_count, total_days, funds):
    
    if funds>=75:
        elf_hire_total = np.random.randint(funds/75)
    else:
        elf_hire_total = 0
    if elf_count == 0:
        elf_hire_total+=1
    total_elves = elf_count + elf_hire_total
    wood_elf_count = np.random.randint(total_elves+1)
    forest_elf_count = np.random.randint(total_elves - wood_elf_count+1)
    mountain_elf_count = total_elves - forest_elf_count - wood_elf_count
    
    return wood_elf_count, forest_elf_count, mountain_elf_count, elf_hire_total

In [5]:
## Bot_elf_game runs the elf game given a bot strategy, such as random_plan_generator above.

def bot_elf_game(bot, total_days):
    day_count = 0
    funds = 0
    profit_tracker = []
    elf_count_tracker = []
    weather_tracker = []
    plan_tracker = []
    elf_count = 12
    while day_count < total_days:
        plan = bot(elf_count, day_count, total_days, funds)
        weather_today = weather()
        weather_tracker.append(weather_today)
        funds += day_profit(weather_today, *plan)
        elf_count = elf_count_adjuster(elf_count, plan[2], plan[3], weather_today)
        elf_count_tracker.append(elf_count)
        
        day_count+=1
        
        
        profit_tracker.append(funds)
    
    return funds, profit_tracker, elf_count_tracker, weather_tracker

In [6]:
## A function to play multiple bots against each other.
## Results for each bot are stored in a dictionary within kwargs

def multibot_elf_game(total_days,**kwargs):
    day_count = 0
    tracker = {}
    bot_start = {'funds':0,'elf_count':12,'profit_tracker':[],'elf_count_tracker':[]}
    for bot in kwargs:
        tracker[kwargs[bot].__name__]=deepcopy(bot_start)
    weather_tracker = []
    while day_count < total_days:
        
        weather_today = weather()
        weather_tracker.append(weather_today)
        
        for bot in kwargs:
            botname = kwargs[bot].__name__
            elf_count = tracker[botname]['elf_count']
            funds = tracker[botname]['funds']
            elf_count_tracker = tracker[botname]['elf_count_tracker']
            profit_tracker = tracker[botname]['profit_tracker']
            plan = kwargs[bot](elf_count, day_count, total_days, funds)
            #print weather_today, botname, plan

            funds += day_profit(weather_today, *plan)

            elf_count = elf_count_adjuster(elf_count, plan[2], plan[3], weather_today)
            elf_count_tracker.append(elf_count)
            profit_tracker.append(funds)

            tracker[botname]['funds'] += day_profit(weather_today, *plan)
            tracker[botname]['elf_count'] = elf_count_adjuster(elf_count, plan[2], plan[3], weather_today)
            tracker[botname]['elf_count_tracker'].append(tracker[botname]['elf_count'])
            tracker[botname]['profit_tracker'].append(tracker[botname]['funds'])
        day_count+=1
        
    results = {}
    for bot in kwargs:
        botname = kwargs[bot].__name__
        results[botname]={}
        results[botname]['funds']=tracker[botname]['funds']
        results[botname]['elfcount']=tracker[botname]['elf_count']
        
    
    return results, weather_tracker



In [8]:
def phil_bot(elf_count, day_count, total_days, funds):
    #in the first third of the game, hire as many elves as possible
    #hit the forest every day
    
    if funds>=75 and day_count<total_days/3:
        elf_hire_total = funds/75
    else:
        elf_hire_total = 0
    total_elves = elf_count + elf_hire_total
    wood_elf_count = 0
    forest_elf_count = total_elves
    mountain_elf_count = 0
    
    return wood_elf_count, forest_elf_count, mountain_elf_count, elf_hire_total

def tms_bot(elf_count, day_count, total_days, funds):
    # a more complex strategy involving elf hires, a proportion of elves going to the mountain and the forest
    if funds>=300 and day_count<=total_days*0.4:
        elf_hire_total = funds/(75)
    else:
        elf_hire_total = 0
    if elf_count == 0 and funds>75:
        elf_hire_total+=1
    total_elves = elf_count + elf_hire_total
    wood_elf_count = 0
    # send 20% of elves to the mountains when you have more than 30 elves
    if total_elves>30:
        forest_elf_count = 5*total_elves/6
        mountain_elf_count = total_elves - forest_elf_count
    else:
        forest_elf_count = total_elves
        mountain_elf_count = 0
    
    #Hail Mary send all elves to the mountain on the last day
    if day_count==total_days-1:
        mountain_elf_count=total_elves
    return wood_elf_count, forest_elf_count, mountain_elf_count, elf_hire_total
    
def woodland_bot(elf_count, day_count, total_days, funds):
    # send all elves to the woods
    if funds>=75 and day_count<total_days/5:
        elf_hire_total = funds/75
    else:
        elf_hire_total=0
    wood_elf_count = elf_count
    forest_elf_count = 0
    mountain_elf_count = 0
    
    return wood_elf_count, forest_elf_count, mountain_elf_count, elf_hire_total

def jon_bot(elf_count, day_count, total_days, funds):
    # In the first half of the game, hire as many elves as possible
    #send 60% of elves to the woods, 10% forest and 30% mountains
    if funds>=75 and day_count<total_days/2:
        elf_hire_total = funds/75
    else:
        elf_hire_total = 0
    total_elves = elf_count + elf_hire_total
    wood_elf_count = total_elves*6/10
    forest_elf_count = total_elves/10
    mountain_elf_count = total_elves-forest_elf_count-wood_elf_count
    
    return wood_elf_count, forest_elf_count, mountain_elf_count, elf_hire_total

#def mountain_bot():
    #all elves to the mountain


In [10]:
## Trying to optimise TMS_bot by bruteforce
## Establish how much time should pass before you stop investing in elves
## Same model could be used for bruteforce of minimum funds for hiring, 
## proportion of elves to send to the mountains, and with what elf population



pwins=[]
for dayfract in range(0,100):
    
    def tms_test_bot(elf_count, day_count, total_days, funds):
        #invest half the profit
        # some elves to forest, some to mountains
        if funds>=300 and day_count<=total_days*dayfract/100:
            elf_hire_total = funds/(75*2)
        else:
            elf_hire_total = 0
        if elf_count == 0 and funds>75:
            elf_hire_total+=1
        total_elves = elf_count + elf_hire_total
        wood_elf_count = 0
        if total_elves>25:
            forest_elf_count = 5*total_elves/6
            mountain_elf_count = total_elves - forest_elf_count
        else:
            forest_elf_count = total_elves
            mountain_elf_count = 0
    
        return wood_elf_count, forest_elf_count, mountain_elf_count, elf_hire_total


    phil_total = []
    tms_total = []
    for trys in range(100):
        game = multibot_elf_game(10,bot1=phil_bot,bot2=tms_test_bot)
        phil_total.append(game[0]['phil_bot']['funds'])
        tms_total.append(game[0]['tms_test_bot']['funds'])
    

    
    phil_wins = 0 
    for game in enumerate(phil_total):
        if game[1]>tms_total[game[0]]:
            phil_wins +=1
    tms_wins=100-phil_wins
    pwins.append(phil_wins)
    
print pwins
print min(pwins)
print "Stop investing after %s percent of the time"%pwins.index(min(pwins))



[92, 81, 86, 88, 92, 86, 95, 88, 86, 91, 91, 84, 86, 92, 94, 90, 88, 88, 81, 82, 91, 89, 88, 87, 83, 87, 89, 90, 88, 92, 90, 88, 89, 94, 89, 87, 87, 85, 90, 88, 64, 60, 64, 71, 67, 62, 66, 61, 70, 56, 46, 55, 54, 58, 48, 60, 48, 50, 58, 56, 53, 59, 65, 50, 56, 50, 55, 48, 54, 59, 54, 54, 50, 58, 43, 46, 55, 53, 53, 53, 64, 55, 64, 70, 59, 59, 56, 61, 56, 58, 80, 78, 76, 71, 73, 80, 74, 71, 74, 78]
43
Stop investing after 74 percent of the time


In [11]:
## Run phil_bot against tms_bot to see who is the ultimate elf-champ

phil_total = []
tms_total = []

for p in range(20000):
    g = multibot_elf_game(10,bot1=phil_bot,bot2=tms_bot, bot3 = jon_bot)
    phil_total.append(g[0]['phil_bot']['funds'])
    tms_total.append(g[0]['tms_bot']['funds'])
   
print np.mean(phil_total)
print np.std(phil_total)
print np.mean(tms_total)
print np.std(tms_total)

print "Phil won %s out of 20000 games" %len([j for i, j in enumerate(phil_total) if j > tms_total[i]]) 

2203.19
759.190887656
3198.17825
1640.38992315
Phil won 2959 out of 20000 games
